# Библиотеки

In [2]:
from zlib import crc32

import numpy as np
import pandas as pd
import scipy.stats as st
import re
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
import scipy.stats as st

# Код генерации выборки

In [3]:
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code)
    tip = rs.randint(low=0, high=3)
    if tip == 0:
        w = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
    if tip == 1:
        w= [1, 1, 1, 1, 1, 0, 0, 0, 0, 0][::-1]
    if tip == 2:
        w = [0]*10
    x = rs.randn(1000, 10)
    y = np.dot(x, w) + rs.randn(1000)*0.0001
    y[100:] += 10*rs.randn(900)*rs.uniform(size = 900)
    y -= np.min(y) - 0.01
    return np.hstack([y.reshape(1000,1), x])

# Генерация выборки для вашей почты

<span style="color:red">
    ВАЖНО!
    Почта, которую укажете ниже и почта с которой Вы отправляете домашнее задание должна совпадать!
    В момент проверки задания алгоритм генерирует выборку на основе почты из анкеты!
</span>

Внимательно проверьте почту для которой выполняется задание!

In [4]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

task

Enter your mail: bogdanov.ai@phystech.edu


{'mail': 'bogdanov.ai@phystech.edu',
 'id': 1162440694,
 'data': array([[18.55654613,  1.08945958,  0.32178445, ...,  2.66738059,
         -0.45650835, -0.35949748],
        [21.21694915, -1.39968299, -1.59509462, ..., -0.03786994,
          1.55498625, -0.66885577],
        [21.89284214,  0.3648031 , -0.64772248, ...,  1.80220501,
          0.96381658, -0.06935707],
        ...,
        [20.50369098, -0.73637306, -0.89386267, ...,  0.60072037,
          1.20160435,  0.23308067],
        [19.9775419 , -0.34561843, -0.66330805, ...,  0.62327406,
         -1.20261697,  0.37706688],
        [12.90259693, -0.86797526,  1.52408046, ..., -0.62932746,
          1.00534656,  0.23907955]])}

# Работа с выборкой

In [5]:
data = task['data']

## Постройте линейную модель Y от X и свободного коэффициента. Проверьте, есть ли гетероскедастичность в выборке с использованием критерия Бройша-Пагана (использовать F-test, см. справку по критерию)

In [6]:
data = pd.DataFrame(data)
column_names = ['Y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']
data.columns = column_names

In [7]:
X = data.drop('Y', axis=1)
y = data['Y']

In [8]:
model = sm.OLS(endog=y, exog=sm.add_constant(X)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     12.81
Date:                Sat, 13 Apr 2024   Prob (F-statistic):           3.30e-21
Time:                        07:05:44   Log-Likelihood:                -3099.7
No. Observations:                1000   AIC:                             6221.
Df Residuals:                     989   BIC:                             6275.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         18.2936      0.171    106.829      0.000      17.958      18.630
X0            -0.2582      0.173     -1.491      0.136      -0.598       0.082
X1            -0.0165      0.168     -0.098      0.922      -0.347       0.314
X2             0.0946      0.171      0.551      0.581      -0.242       0.431
X3            -0.1605      0.174     -0.922      0.357      -0.502       0.181
X4            -0.2017      0.177     -1.141      0.254      -0.548       0.145
X5             0.8524      0.171      4.994      0.000       0.517       1.187
X6             0.8922      0.172      5.191      0.000       0.555       1.230
X7             0.7475      0.165      4.527      0.000       0.423       1.072
X8             0.9053      0.169      5.356      0.000       0.574       1.237
X9             0.8212      0.180      4.567      0.000       0.468       1.174
==============================================================================
Omnibus:                       90.244   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              300.470
Skew:                           0.403   Prob(JB):                     5.67e-66
Kurtosis:                       5.561   Cond. No.                         1.19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
new_X = X
new_X['Bias'] = 1

In [10]:
het_breuschpagan(model.resid, new_X)[3] >= 0.05

True

## Оптимальное значение lambda для преобразования Бокса-Кокса на переменную Y (использовать scipy.stats)

In [11]:
_, l = st.boxcox(y)
print(f'Оптимальная lambda = {l.round(3)}')

Оптимальная lambda = 0.883


## Постройте линейную модель Y от X и свободного коэффициента для первых 100 элементов выборки. Сколько переменных являются избыточными согласно t-критерию? Поправку на множественность гипотез проигнорировать

In [12]:
model100 = sm.OLS(endog=y[:100], exog=sm.add_constant(X[:100])).fit()
model100.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.812e+09
Date:                Sat, 13 Apr 2024   Prob (F-statistic):               0.00
Time:                        07:05:45   Log-Likelihood:                 786.38
No. Observations:                 100   AIC:                            -1551.
Df Residuals:                      89   BIC:                            -1522.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
X0         -7.746e-06    9.4e-06     -0.824      0.412   -2.64e-05    1.09e-05
X1         -1.715e-05   9.29e-06     -1.846      0.068   -3.56e-05    1.31e-06
X2          5.147e-06      1e-05      0.514      0.608   -1.47e-05     2.5e-05
X3         -4.285e-06   1.28e-05     -0.336      0.738   -2.97e-05    2.11e-05
X4          1.485e-05   1.31e-05      1.133      0.260   -1.12e-05    4.09e-05
X5             1.0000   9.72e-06   1.03e+05      0.000       1.000       1.000
X6             1.0000   1.07e-05   9.39e+04      0.000       1.000       1.000
X7             1.0000   9.88e-06   1.01e+05      0.000       1.000       1.000
X8             1.0000   9.99e-06      1e+05      0.000       1.000       1.000
X9             1.0000   1.12e-05   8.89e+04      0.000       1.000       1.000
Bias          18.0253   1.01e-05   1.78e+06      0.000      18.025      18.025
==============================================================================
Omnibus:                        2.332   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.312   Jarque-Bera (JB):                1.879
Skew:                          -0.328   Prob(JB):                        0.391
Kurtosis:                       3.142   Cond. No.                         1.89
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
print(f'Количество избыточных переменных = {len(model100.pvalues[model100.pvalues >= 0.05])}')

Количество избыточных переменных = 5


## Имеет ли смысл данная модель согласно F-критерию?

In [17]:
model100.f_test('X0 = X1 = X2 = X3 = X4 = X5 = X6 = X7 = X8 = X9 = 0')
print(model100.f_pvalue < 0.05)

True


## Можно ли обнулить первые четыре переменные согласно критерию Вальда?

In [18]:
model100.wald_test('X0 = X1 = X2 = X3 = 0', scalar=False)
print(model100.f_pvalue >= 0.05)

False
